In [1]:
# Based on Zhang/Oxford paper and github/shilewenuw implementation

In [2]:
from Model import Model
import pandas as pd

In [23]:
dfr = pd.read_csv("new_returns.csv") # returns
dff = pd.read_csv("new_features.csv") # features

In [25]:
dff.shape

(20153, 312)

In [4]:
close_cols = [colname for colname in dff.columns if colname.split()[1] == "close"]
close_cols

['AdM close',
 'AL close',
 'A close',
 'GO close',
 'JS close',
 'KvW close',
 'KA close',
 'SA close',
 'S close',
 'Ve close',
 'Vi close',
 'Z close']

In [24]:
df_close = dff[close_cols].copy()
df_close.rename(columns={colname: colname.split()[0] for colname in df_close.columns}, inplace=True)
df_close.head()

,AdM,AL,A,GO,JS,KvW,KA,SA,S,Ve,Vi,Z
0,229.912597,1048.195089,5258.509097,785.749481,69.970392,21.790732,3065.433564,6791.196084,30.613892,61.681587,3877.559631,521.150292
1,222.457725,1026.394270,5125.827632,767.167567,68.744350,20.976506,3004.030094,6612.089249,29.557249,60.201578,3843.123472,499.684147
2,215.705123,1025.809013,5080.297423,758.672978,69.116046,21.017217,2963.956251,6939.832461,30.060412,60.136283,3805.161980,499.684147
3,219.054413,1035.758381,5120.078868,763.318457,69.215905,21.159707,2969.127069,6837.273361,30.146668,60.223342,3811.257294,492.358399
4,213.922436,1022.882729,5066.500391,753.363860,68.486382,20.966328,2947.366541,6743.632443,30.132292,59.374513,3777.989667,478.388368


In [6]:
model = Model()

In [7]:
coeffs_tm2 = model.get_allocations(df_close[:-2])

1/1 [==============================] - 1s 544ms/step


In [8]:
coeffs_tm2

array([0.2859278 , 0.03623775, 0.18083054, 0.08025122, 0.05769403,
       0.02290204, 0.05734631, 0.06026483, 0.11917441, 0.03499258,
       0.02101563, 0.04336284], dtype=float32)

In [9]:
sum(coeffs_tm2)

0.999999962747097

In [10]:
model = Model()

In [11]:
coeffs_tm1 = model.get_allocations(df_close[:-1])

1/1 [==============================] - 1s 544ms/step


In [12]:
coeffs_tm1

array([0.17749152, 0.08226611, 0.02629446, 0.08284896, 0.11644927,
       0.03614454, 0.04697401, 0.1573087 , 0.14407232, 0.02979228,
       0.04659253, 0.05376534], dtype=float32)

In [13]:
import numpy as np
np.round(coeffs_tm1 - coeffs_tm2,2)

array([-0.11,  0.05, -0.15,  0.  ,  0.06,  0.01, -0.01,  0.1 ,  0.02,
       -0.01,  0.03,  0.01], dtype=float32)

In [14]:
df_close[-2:]

,AdM,AL,A,GO,JS,KvW,KA,SA,S,Ve,Vi,Z
1725,229.043523,1045.770874,5221.756555,757.077607,71.917289,19.020708,2887.268990,12099.843032,32.793155,54.264949,3003.830105,437.004896
1726,231.478717,1059.066738,5253.608610,760.743514,72.627394,19.097151,2916.247762,12264.722412,34.133605,55.387076,3020.917053,443.717365


In [15]:
np.sum(( np.multiply(df_close[-1:].to_numpy()[0], coeffs_tm1) - np.multiply(df_close[-2:-1].to_numpy()[0], coeffs_tm2)))

480.40413651493293

For Loop

In [16]:
iters = 10

In [17]:
old_coeffs = np.ones(12) / 12.0

In [18]:
old_coeffs

array([0.08333333, 0.08333333, 0.08333333, 0.08333333, 0.08333333,
       0.08333333, 0.08333333, 0.08333333, 0.08333333, 0.08333333,
       0.08333333, 0.08333333])

In [19]:
profits = []

In [20]:
for i in range(iters):
    print(f"Iteration: {i+1}")
    model = Model()
    next_index = i-iters
    coeffs = model.get_allocations(df_close[i:next_index])
    hourly_profit = np.sum(( np.multiply(df_close.iloc[next_index-1].to_numpy()[0], old_coeffs) - np.multiply(df_close.iloc[next_index].to_numpy()[0], coeffs)))
    print(f"Profit = {hourly_profit}")
    profits.append(hourly_profit)
    print(f"Cumulative Profit = {sum(profits)}")
    old_coeffs = coeffs
    print("---------------")

Iteration: 1


1/1 [==============================] - 1s 696ms/step
Profit = -1.5152157855809776
Cumulative Profit = -1.5152157855809776
---------------
Iteration: 2
1/1 [==============================] - 1s 742ms/step
Profit = 1.1093559265136719
Cumulative Profit = -0.4058598590673057
---------------
Iteration: 3
1/1 [==============================] - 1s 526ms/step
Profit = -0.6223340034484863
Cumulative Profit = -1.028193862515792
---------------
Iteration: 4
1/1 [==============================] - 1s 544ms/step
Profit = 1.596395492553711
Cumulative Profit = 0.5682016300379189
---------------
Iteration: 5
1/1 [==============================] - 1s 842ms/step
Profit = -1.082322120666504
Cumulative Profit = -0.514120490628585
---------------
Iteration: 6
1/1 [==============================] - 1s 512ms/step
Profit = -1.7046318054199219
Cumulative Profit = -2.218752296048507
---------------
Iteration: 7
1/1 [==============================] - 1s 522ms/step
Profit = 1.2987799644470215
Cumulative Profit = -

In [21]:
profits

[-1.5152157855809776,
 1.1093559,
 -0.622334,
 1.5963955,
 -1.0823221,
 -1.7046318,
 1.29878,
 0.18940973,
 0.59527063,
 -2.4351878]

In [22]:
sum(profits)

-2.570479781705245